In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread

Using TensorFlow backend.


In [2]:
train = pd.read_json('../_RawData/train.json/data/processed/train.json')

In [3]:
def get_images(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    images2 = []
    images3 = []
    images4 = []
    images5 = []
    images6 = []
    images7 = []
    images8 = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
        
        rgb2 = np.rot90(rgb, k = 1, axes = (0, 1))
        images2.append(rgb2)
                
        rgb3 = np.rot90(rgb, k = 2, axes = (0, 1))
        images3.append(rgb3)
                
        rgb4 = np.rot90(rgb, k = 3, axes = (0, 1))
        images4.append(rgb4)
                
        rgb5 = np.flipud(rgb)
        images5.append(rgb5)
        
        rgb6 = np.flipud(rgb2)
        images6.append(rgb6)
                
        rgb7 = np.flipud(rgb3)
        images7.append(rgb7)
                
        rgb8 = np.flipud(rgb4)
        images8.append(rgb8)
        
    images = np.array(images)
    images2 = np.array(images2)
    images3 = np.array(images3)
    images4 = np.array(images4)
    images5 = np.array(images5)
    images6 = np.array(images6)
    images7 = np.array(images7)
    images8 = np.array(images8)
    
    images = np.append(images, images2, axis = 0)
    images = np.append(images, images3, axis = 0)
    images = np.append(images, images4, axis = 0)
    images = np.append(images, images5, axis = 0)
    images = np.append(images, images6, axis = 0)
    images = np.append(images, images7, axis = 0)
    images = np.append(images, images8, axis = 0)
        
    return np.array(images)

In [4]:
X = get_images(train)

In [5]:
X.shape

(12832, 75, 75, 3)

In [6]:
y = to_categorical(train.is_iceberg.values,num_classes=2)
y = np.append(y, y, axis=0)
y = np.append(y, y, axis=0)
y = np.append(y, y, axis=0)

In [7]:
y.shape

(12832, 2)

In [8]:
Xtr, Xv, ytr, yv = train_test_split(X, y, shuffle=False, test_size=0.20)

In [9]:
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [10]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(2, (3, 3), activation='relu'))
    model.add(GlobalAveragePooling2D())
    # 4x4x2
    model.add(Activation('softmax'))
    
    return model

In [11]:
# Create the model and compile
model = create_model()
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 75, 75, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 77, 77, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
__________

In [13]:
init_epo = 0
num_epo = 10
end_epo = init_epo + num_epo

In [ ]:
print ('lr = {}'.format(K.get_value(model.optimizer.lr)))
history = model.fit(Xtr, ytr, validation_data=(Xv, yv), batch_size=32, epochs=end_epo, initial_epoch=init_epo)
init_epo += num_epo
end_epo = init_epo + num_epo

lr = 9.999999747378752e-05
Train on 10265 samples, validate on 2567 samples
Epoch 1/10
10265/10265 [==============================] - 13s 1ms/step - loss: 0.4164 - acc: 0.7861 - val_loss: 2.0585 - val_acc: 0.5423
Epoch 2/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.2975 - acc: 0.8530 - val_loss: 0.4462 - val_acc: 0.7826
Epoch 3/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.2630 - acc: 0.8761 - val_loss: 0.3375 - val_acc: 0.8337
Epoch 4/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.2420 - acc: 0.8867 - val_loss: 0.3025 - val_acc: 0.8551
Epoch 5/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.2112 - acc: 0.9029 - val_loss: 0.3226 - val_acc: 0.8446
Epoch 6/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.1804 - acc: 0.9245 - val_loss: 0.2811 - val_acc: 0.8695
Epoch 7/10
10265/10265 [==============================] - 11s 1ms/step - loss: 0.1538 - acc: 0.9397 - 

In [ ]:
test = pd.read_json('../_RawData/test.json/data/processed/test.json')
Xtest = get_images(test)
test_predictions = model.predict_proba(Xtest)
submission = pd.DataFrame({'id': test['id'], 'is_iceberg': test_predictions[:, 1]})
submission.to_csv('submission3.csv', index=False)